GRBL Settings

In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
%%file GRBL.py
import serial
class GRBL(object):
    BAUDRATE = 115200
    
    def __init__(self, port):
        self.serial = serial.Serial(port=port,
                                    baudrate=GRBL.BAUDRATE,
                                    timeout=0.10)
        
    def write(self, command_line=""):
        self.serial.flushInput()
        self.serial.write("\n".encode())
        self.serial.write("{cmd}\n".format(cmd=command_line).encode())
        
    def read(self, multiline=True):
        if multiline:
            responses = self.serial.readlines()
            responses = [response.decode().strip() for response in responses]
            return responses
        else:
            response = self.serial.readline()
            return response.decode().strip()
 
    def cmd(self, command_line, resp=True, multiline=True):
        self.write(command_line)
        if resp:
            return self.read(multiline=multiline)
        return None
    
    def reset(self):
        """ https://github.com/gnea/grbl/wiki/Grbl-v1.1-Commands#grbl-v11-realtime-commands
        """
        ret = self.cmd("\x18")
        assert(ret[-1]=='ok')
        
    def sleep(self):
        """ https://github.com/gnea/grbl/wiki/Grbl-v1.1-Commands#slp---enable-sleep-mode
        """
        ret = self.cmd("$SLP")
        assert(ret[-1]=='ok')
    
    @property
    def status(self):
        """
        """
        ret = self.cmd("?")
        assert(ret[-1]=='ok')
        return ret[1]
        
    def kill_alarm(self):
        """ https://github.com/gnea/grbl/wiki/Grbl-v1.1-Commands#x---kill-alarm-lock
        """
        ret = self.cmd("$X")
        assert(ret[-1]=='ok')
        
    def home(self):
        """ https://github.com/gnea/grbl/wiki/Grbl-v1.1-Commands#h---run-homing-cycle
        """
        self.write("$H")
        assert(ret[-1]=='ok')
        
# https://github.com/gnea/grbl/wiki/Grbl-v1.1-Configuration#---view-grbl-settings
settings = [
    ("$0", "step_pulse"),
    ("$1", "step_idle_delay"),
    ("$2", "step_port_invert"),
    ("$3", "direction_port_invert"),
    ("$4", "step_enable_invert"),
    ("$5", "limit_pin_invert"),
    ("$6", "probe_pin_invert"),
    ("$10", "status_report"),
    ("$11", "junction_deviation"),
    ("$12", "arc_tolerance"),
    ("$13", "report_inches"),
    ("$20", "soft_limits"),
    ("$21", "hard_limits"),
    ("$22", "homing_cycle"),
    ("$23", "homing_dir_invert"),
    ("$24", "homing_feed"),
    ("$25", "homing_seek"),
    ("$26", "homing_debounce"),
    ("$27", "homing_pull_off"),
    ("$30", "max_spindle_speed"),
    ("$31", "min_spindle_speed"),
    ("$32", "laser_mode"),
    ("$100", "x_steps_mm"),
    ("$101", "y_steps_mm"),
    ("$102", "z_steps_mm"),
    ("$110", "x_max_rate"),
    ("$111", "y_max_rate"),
    ("$112", "z_max_rate"),
    ("$120", "x_acceleration"),
    ("$121", "y_acceleration"),
    ("$122", "z_acceleration"),
    ("$130", "x_travel"),
    ("$131", "y_travel"),
    ("$132", "z_travel"),
    ]

def grbl_getter_generator(cmd):
    def grbl_getter(self):
        config = self.cmd("$$", resp=True, multiline=True)
        for config_line in config:
            if config_line.startswith("$"):
                key, value = config_line.split("=")
                if key == cmd:
                    return float(value)
        return None
    return grbl_getter
    
def grbl_setter_generator(cmd):
    def grbl_setter(self, value):
        set_cmd = "{cmd}={value}".format(cmd=cmd, value=value)
        ret = self.cmd(set_cmd, resp=True, multiline=False)
        print(ret)
        
    return grbl_setter

for setting in settings:
    cmd = setting[0]
    name = setting[1]
    
    setter = grbl_setter_generator(cmd)
    getter = grbl_getter_generator(cmd)
    
    prop = property(fget=getter,
                    fset=setter,
                    doc=" ".join(name.split("_")))
    
    setattr(GRBL, name, prop)
    
# https://github.com/gnea/grbl/wiki/Grbl-v1.1-Commands#---view-gcode-parameters
gcode_parameters = [
    "G54",
    "G55",
    "G56",
    "G57",
    "G58",
    "G59",
    "G28",
    "G30",
    "G92",
    "TLO",
    "PRB",
    ]


def gcode_param_gen(parameter):
    def gcode_param(self):
        gcode_parameters = self.cmd("$#") # View gcode parameters
        for gcode_parameter in gcode_parameters: 
            if parameter in gcode_parameter:
                _, value = gcode_parameter.split(":")
                value = value.strip("]")
                values = value.split(",")
                values = [float(value) for value in values]
                
                return values
        return None
    return gcode_param


for parameter in gcode_parameters:
    fcn = gcode_param_gen(parameter)
    prop = property(fget=fcn)
    setattr(GRBL, parameter, prop)

Overwriting GRBL.py


In [3]:
%aimport GRBL

In [5]:
grbl = GRBL.GRBL("/dev/cnc_3018")

In [8]:
grbl.reset()

In [9]:
# Metric
grbl.cmd("G21")
# Relative
grbl.cmd("G91")

['ok', 'ok']

In [10]:
grbl.cmdd("$$")

['ok',
 '$0=10',
 '$1=25',
 '$2=0',
 '$3=0',
 '$4=0',
 '$5=0',
 '$6=0',
 '$10=1',
 '$11=0.010',
 '$12=0.002',
 '$13=0',
 '$20=0',
 '$21=0',
 '$22=0',
 '$23=0',
 '$24=25.000',
 '$25=500.000',
 '$26=250',
 '$27=1.000',
 '$30=1000',
 '$31=0',
 '$32=1',
 '$100=250.000',
 '$101=250.000',
 '$102=250.000',
 '$110=500.000',
 '$111=500.000',
 '$112=500.000',
 '$120=10.000',
 '$121=10.000',
 '$122=10.000',
 '$130=200.000',
 '$131=200.000',
 '$132=200.000',
 'ok']

In [11]:
import configparser

In [13]:
cfg = configparser.ConfigParser()

In [14]:
cnc3018 = cfg.add_section("CNC_3018")

In [19]:
# https://github.com/gnea/grbl/issues/123#issuecomment-318474119
cnc3018_defaults = """$0	10	(step pulse, usec)
$1	25	(step idle delay, msec)
$2	0	(step port invert mask:00000000)
$3	5	(dir port invert mask:00000101)
$4	0	(step enable invert, bool)
$5	0	(limit pins invert, bool)
$6	0	(probe pin invert, bool)
$10	3	(status report mask:00000011)
$11	0.010	(junction deviation, mm)
$12	0.002	(arc tolerance, mm)
$13	0	(report inches, bool)
$20	0	(soft limits, bool)
$21	1	(hard limits, bool); 1 Enables the limit switches; 0 Disables them
$22	0	(homing cycle, bool)
$23	0	(homing dir invert mask:00000000)
$24	25.000	(homing feed, mm/min)
$25	500.000	(homing seek, mm/min)
$26	250	(homing debounce, msec)
$27	1.000	(homing pull-off, mm)
$100	800.000	(x, step/mm)
$101	800.000	(y, step/mm)
$102	800.000	(z, step/mm)
$110	800.000	(x max rate, mm/min)
$111	800.000	(y max rate, mm/min)
$112	500.000	(z max rate, mm/min)
$120	10.000	(x accel, mm/sec^2)
$121	10.000	(y accel, mm/sec^2)
$122	10.000	(z accel, mm/sec^2)
$130	200.000	(x max travel, mm)
$131	200.000	(y max travel, mm)
$132	200.000	(z max travel, mm)"""

In [28]:
for line in cnc3018_defaults.splitlines():
    setting, value, comment = line.split("\t")
    cfg["CNC_3018"][setting] = value

In [31]:
with open("grbl_config.ini", "w") as fid:
    cfg.write(fid)

In [36]:
cfg = configparser.ConfigParser()
cfg.read("grbl_config.ini")

['grbl_config.ini']

In [39]:
machine_settings = cfg["CNC_3018"]

In [48]:
for setting, value in machine_settings.items():
    grbl.cmd("{}={}".format(setting, value))

In [49]:
grbl.cmd("$$")

['ok',
 '$0=10',
 '$1=25',
 '$2=0',
 '$3=5',
 '$4=0',
 '$5=0',
 '$6=0',
 '$10=3',
 '$11=0.010',
 '$12=0.002',
 '$13=0',
 '$20=0',
 '$21=1',
 '$22=0',
 '$23=0',
 '$24=25.000',
 '$25=500.000',
 '$26=250',
 '$27=1.000',
 '$30=1000',
 '$31=0',
 '$32=1',
 '$100=800.000',
 '$101=800.000',
 '$102=800.000',
 '$110=800.000',
 '$111=800.000',
 '$112=500.000',
 '$120=10.000',
 '$121=10.000',
 '$122=10.000',
 '$130=200.000',
 '$131=200.000',
 '$132=200.000',
 'ok']

In [51]:
grbl.reset()

In [52]:
grbl.cmd("$$")

['ok',
 '$0=10',
 '$1=25',
 '$2=0',
 '$3=5',
 '$4=0',
 '$5=0',
 '$6=0',
 '$10=3',
 '$11=0.010',
 '$12=0.002',
 '$13=0',
 '$20=0',
 '$21=1',
 '$22=0',
 '$23=0',
 '$24=25.000',
 '$25=500.000',
 '$26=250',
 '$27=1.000',
 '$30=1000',
 '$31=0',
 '$32=1',
 '$100=800.000',
 '$101=800.000',
 '$102=800.000',
 '$110=800.000',
 '$111=800.000',
 '$112=500.000',
 '$120=10.000',
 '$121=10.000',
 '$122=10.000',
 '$130=200.000',
 '$131=200.000',
 '$132=200.000',
 'ok']

In [53]:
grbl.cmd("M3 S1")

['ok', 'ok']